Importing libraries and setting up file paths

In [34]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

path="data/"
sub={'production_meter/':['average_power(kw)/'], 'weather_station/':['temperature(f)/','irradiance(w_per_m2)/']}
med='15_minutes/'
fin=['12_23_2019-12_31_2019', '01_01_2020-01_31_2020','02_01_2020-02_29_2020','03_01_2020-03_31_2020','04_01_2020-04_30_2020','05_01_2020-05_31_2020','06_01_2020-06_30_2020','07_01_2020-07_31_2020','08_01_2020-08_31_2020','09_01_2020-09_30_2020','10_01_2020-10_31_2020','11_01_2020-11_30_2020','12_01_2020-12_26_2020']
end='.csv'

#these will form the full path name of the raw data files

Importing the raw data files

In [35]:
csvs={}
for i in sub: #going through each source for the data
    for j in sub[i]: #going though each category in each source
        csvs[j]=pd.DataFrame()
        for k in fin: #going though each month available
            t=pd.read_csv(path+i+j+med+k+end, index_col='Site Time', parse_dates=['Site Time']) #reading a full path name for one csv file, with the timestamps read as datettime objects for the index
            csvs[j]=pd.concat([csvs[j], t])
power, temp, irradiance=csvs['average_power(kw)/'], csvs['temperature(f)/'], csvs['irradiance(w_per_m2)/']

Command to only keep hourly timestamps

In [36]:
power=power[power.index.minute==0] #this line is run when only hourly increments are wanted

Calculating a new cyclical time feature from the timestamps

In [37]:
#extracting a cyclical time value from the timestamps
from astral.geocoder import lookup, database
import datetime
city = lookup("Hartford", database()) #closest city to Windsor CT, the location of the solar field
from astral.sun import sun
power['time']=power.index.copy() #copying the timestamp index, to be modified below
def conversion(time): #function for converting a single timestamp
    s = sun(city.observer, date=datetime.date(time.year, time.month, time.day))
    sunrise=pd.to_datetime(s['sunrise'].__str__()).tz_convert('US/Eastern').tz_localize(None) #getting the sunrise time, then converting to the correct timezone and then removing timezone info
    sunset=pd.to_datetime(s['sunset'].__str__()).tz_convert('US/Eastern').tz_localize(None) #getting the sunset time
    noon=pd.to_datetime(s['noon'].__str__()).tz_convert('US/Eastern').tz_localize(None) #getting the noon time
    half=(noon-sunrise).total_seconds()
    if (time-sunrise).days<0 or (sunset-time).days<0:
        return 0 #non daylight times will be set to 0
    if (time-noon).days<0:
        return 1-((noon-time).total_seconds()/half) 
    return 1-((time-noon).total_seconds()/half)
power['time']=power['time'].apply(conversion) 

Replacing anomalous temperature values with the median

In [38]:
temp['Weather station device temperature '][temp['Weather station device temperature ']<0]=np.nan #the anomalous values, which are less than 0 degrees f, are set to NaN
temp.fillna(temp['Weather station device temperature '].median(), inplace=True) #the NaN values are replaced with the median

Creating all power data features

In [39]:
one_ahead_power=power.shift(-1).copy() #creating the target data from shifting the current power ahead one
one_behind_power=power.shift(1).copy().drop(labels='time', axis=1) #creating a new feature from shifting current power behind one
two_behind_power=power.shift(2).copy().drop(labels='time', axis=1) #creating a new feature from shifting current power behind two
one_ahead_power=one_ahead_power.rename(columns={'Production meter active power Kilowatts':'one ahead power Kilowatts'})
one_behind_power=one_behind_power.rename(columns={'Production meter active power Kilowatts':'one behind power Kilowatts'})
two_behind_power=two_behind_power.rename(columns={'Production meter active power Kilowatts':'two behind power Kilowatts'})
power=power.drop('time', axis=1) #the only time column now is in one_ahead_power, reflecting the time for the target values

Merging all power data into one DataFrame

In [40]:
power=two_behind_power.merge(one_behind_power, on='Site Time').merge(power, on='Site Time').merge(one_ahead_power, on='Site Time') #merging all the power data and aligning them along the timestamp

Rows with missing values are dropped

In [41]:
#removing missing values
temp=temp.dropna()
irradiance=irradiance.dropna()
power=power.dropna()
power=power.rename(columns={'Production meter active power Kilowatts':'current power Kilowatts'})

Data is converted to numeric types

In [42]:
power['current power Kilowatts']=pd.to_numeric(power['current power Kilowatts'].astype(str).str.replace(',','')) #the issue is that some entries have commas, like 1,000, so first they are converted to strings, then the commas are removed before conversion to numbers
power['two behind power Kilowatts']=pd.to_numeric(power['two behind power Kilowatts'].astype(str).str.replace(',',''))
power['one behind power Kilowatts']=pd.to_numeric(power['one behind power Kilowatts'].astype(str).str.replace(',',''))
power['one ahead power Kilowatts']=pd.to_numeric(power['one ahead power Kilowatts'].astype(str).str.replace(',',''))
irradiance['POA Watts/meter²'], irradiance['GHI Watts/meter²']=pd.to_numeric(irradiance['POA Watts/meter²'].astype(str).str.replace(',','')), pd.to_numeric(irradiance['GHI Watts/meter²'].astype(str).str.replace(',',''))

Power and irradiance values less than 0 are set to 0

In [43]:
power[power<0]=0
irradiance[irradiance<0]=0

Merging all data columns into one large DataFrame with all the data

In [44]:
site_data=power.merge(temp, on='Site Time').merge(irradiance, on='Site Time')

The following line is run when only daytime values are wanted

In [28]:
site_data=site_data[site_data['time']>0]

Description of current large DataFrame containing all data

In [45]:
site_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8218 entries, 2019-12-23 10:00:00 to 2020-12-26 22:00:00
Data columns (total 10 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   two behind power Kilowatts                              8218 non-null   float64
 1   one behind power Kilowatts                              8218 non-null   float64
 2   current power Kilowatts                                 8218 non-null   float64
 3   one ahead power Kilowatts                               8218 non-null   float64
 4   time                                                    8218 non-null   float64
 5   Weather station module temperature                      8218 non-null   float64
 6   Weather station ambient temperature Degrees Fahrenheit  8218 non-null   float64
 7   Weather station device temperature                      8218 non-null   float64
 8   PO

Splitting the data into a training and test set, as well as input and target sets

In [46]:
X=site_data.drop(columns=['one ahead power Kilowatts'], axis=1) #dropping the target column from input dataset
y=site_data['one ahead power Kilowatts'].copy() #extracting the target column to form target dataset
ratio=.8 #what percentage of the entire data the training set consists of
X_train, X_test, y_train, y_test=X[:int(ratio*len(X))], X[int(ratio*len(X)):], y[:int(ratio*len(y))], y[int(ratio*len(y)):]

Normalization of the input data by column into a fixed range -1 to 1

In [47]:
from sklearn.preprocessing import MinMaxScaler

def scaling(X_train, X_test):
    scalers={}
    for i in X_train.columns: #going through each column in the input
        scaler=MinMaxScaler(feature_range=(-1,1)) #creating the scaler
        X_train[i]=scaler.fit_transform(X_train[i].to_frame()) #converting each column
        scalers[i]=scaler #saving each scaler with its column name
        X_test[i]=scaler.transform(X_test[i].to_frame()) #transforming the test set column with the scaler fitted to the training set
    return X_train, X_test, scalers

X_train, X_test, xscalers=scaling(X_train, X_test)

Writing the preprocessed datasets to csvs

In [48]:
X_train.to_csv('X_train')
X_test.to_csv('X_test')
y_train.to_csv('y_train')
y_test.to_csv('y_test')

Serializing the scaler for the cyclical time value so that the scaling done on this column can later be reversed

In [49]:
import joblib

joblib.dump(xscalers['time'], 'time_transformer') #saving the scaler to a file called 'time_transformer'

['time_transformer']